In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
plt.style.use('dark_background')

Using TensorFlow backend.


# Load Dataset

In [2]:
x_train = np.load('dataset/x_train.npy').astype(np.float32)
y_train = np.load('dataset/y_train.npy').astype(np.float32)
x_val = np.load('dataset/x_val.npy').astype(np.float32)
y_val = np.load('dataset/y_val.npy').astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(2586, 26, 34, 1) (2586, 1)
(288, 26, 34, 1) (288, 1)


# Data Augmentation

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x=x_train, y=y_train,
    batch_size=32,
    shuffle=True
)

val_generator = val_datagen.flow(
    x=x_val, y=y_val,
    batch_size=32,
    shuffle=False
)

# Build Model

In [7]:
inputs = Input(shape=(26, 34, 1))

net = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Flatten()(net)

net = Dense(512)(net)
net = Activation('relu')(net)
net = Dense(1)(net)
outputs = Activation('sigmoid')(net)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 26, 34, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 34, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 17, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 17, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 8, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 4, 128)         0         
__________

# Train

In [6]:
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

model.fit_generator(
    train_generator, epochs=50, validation_data=val_generator,
    callbacks=[
        ModelCheckpoint('models/%s.h5' % (start_time), monitor='val_acc', save_best_only=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05)
    ]
)

Epoch 1/50
81/81 [==============================] - 4s 54ms/step - loss: 0.4922 - acc: 0.7573 - val_loss: 0.2625 - val_acc: 0.9167

Epoch 00001: val_acc improved from -inf to 0.91667, saving model to models/2018_12_17_22_58_35.h5
Epoch 2/50
81/81 [==============================] - 4s 44ms/step - loss: 0.2703 - acc: 0.8950 - val_loss: 0.1172 - val_acc: 0.9653

Epoch 00002: val_acc improved from 0.91667 to 0.96528, saving model to models/2018_12_17_22_58_35.h5
Epoch 3/50
81/81 [==============================] - 4s 45ms/step - loss: 0.1702 - acc: 0.9332 - val_loss: 0.0950 - val_acc: 0.9618

Epoch 00003: val_acc did not improve from 0.96528
Epoch 4/50
81/81 [==============================] - 4s 48ms/step - loss: 0.1452 - acc: 0.9482 - val_loss: 0.0695 - val_acc: 0.9722

Epoch 00004: val_acc improved from 0.96528 to 0.97222, saving model to models/2018_12_17_22_58_35.h5
Epoch 5/50
81/81 [==============================] - 3s 42ms/step - loss: 0.1268 - acc: 0.9579 - val_loss: 0.0532 - val_acc


Epoch 00042: val_acc did not improve from 1.00000
Epoch 43/50
81/81 [==============================] - 3s 41ms/step - loss: 0.0087 - acc: 0.9965 - val_loss: 0.0037 - val_acc: 1.0000

Epoch 00043: val_acc did not improve from 1.00000
Epoch 44/50
81/81 [==============================] - 3s 41ms/step - loss: 0.0065 - acc: 0.9981 - val_loss: 0.0030 - val_acc: 1.0000

Epoch 00044: val_acc did not improve from 1.00000
Epoch 45/50
81/81 [==============================] - 3s 40ms/step - loss: 0.0034 - acc: 0.9988 - val_loss: 0.0027 - val_acc: 1.0000

Epoch 00045: val_acc did not improve from 1.00000
Epoch 46/50
81/81 [==============================] - 3s 40ms/step - loss: 0.0049 - acc: 0.9981 - val_loss: 0.0038 - val_acc: 0.9965

Epoch 00046: val_acc did not improve from 1.00000
Epoch 47/50
81/81 [==============================] - 3s 40ms/step - loss: 0.0051 - acc: 0.9985 - val_loss: 0.0028 - val_acc: 1.0000

Epoch 00047: val_acc did not improve from 1.00000
Epoch 48/50
81/81 [===============